In [1]:
import numpy as np
import pandas as pd
import matplotlib as mlt
import matplotlib.pyplot as plt
import seaborn as sns
import re
from urllib.parse import urljoin

from matplotlib import font_manager, rc   
font_name = font_manager.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

pd.options.display.float_format = "{:,.2f}".format

# 교통량/ 사고다발지역 (Traffic)
## 2. t2 : 도로교통공단_전국_보행노인사고다발지_2017년_

In [2]:
t2 = pd.read_csv("C:/pratice/exam(middle)/datas/t2.csv",encoding="euc-kr") #정부공공데이터:encoding="euc-kr"
t2_1 = t2[["다발지명", "발생건수"]]
t2_1.head()

,다발지명,발생건수
0,부산광역시 부산진구 부전동(부전지구대 부근),5
1,부산광역시 부산진구 가야동(가야고앞 부근),4
2,부산광역시 부산진구 양정동(양정교차로 부근),4
3,대구광역시 서구 내당동(감삼역 부근),4
4,대구광역시 남구 대명동(대구광역시종합복지회관 부근),7


In [3]:
t2_1['서울'] = t2_1['다발지명'].str.split(' ').str[0]
t2_1 = t2_1[t2_1['서울'] == '서울특별시'] 
t2_1['구'] = t2_1['다발지명'].str.split(' ').str[1]
t2_1['동-'] = t2_1['다발지명'].str.split(' ').str[2]
t2_1['동'] = t2_1['동-'].str.split('(').str[0]
t2_1 = t2_1[["구", "동", "발생건수"]] 
t2_1.head()

C:\Users\wooseok-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,구,동,발생건수
6,도봉구,도봉동,2
7,노원구,상계동,4
8,노원구,상계동,4
9,영등포구,당산동1가,3
10,영등포구,대림동,3


In [5]:
t2_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 6 to 238
Data columns (total 3 columns):
구       142 non-null object
동       142 non-null object
발생건수    142 non-null int64
dtypes: int64(1), object(2)
memory usage: 4.4+ KB


In [23]:
# 같은 동끼리 묶기
# 하나라도 묶으면 데이터 프레임
t2_2 = t2_1.groupby([t2_1.구,t2_1.동]).sum()[["발생건수"]]
t2_2.head()

발생건수
구   동        
강남구 도곡동     3
강동구 고덕동     3
    길동      4
    명일동    10
    성내동     3

In [26]:
# 컬럼이름 변경
t2_2.rename(columns={
    t2_2.columns[0]:"노인사고발생건수"
}, inplace=True)

t2_2.head()

노인사고발생건수
구   동            
강남구 도곡동         3
강동구 고덕동         3
    길동          4
    명일동        10
    성내동         3

## 3. t3 : 도로교통공단_전국_보행어린이사고다발지_2017년_

In [14]:
t3 = pd.read_csv("data/pj/t3.csv",encoding="euc-kr") #정부공공데이터:encoding="euc-kr"
t3_1 = t3[["다발지명", "발생건수"]]
t3_1.head()

,다발지명,발생건수
0,경상남도 창원시 진해구 청안동(청안공원 부근),3
1,경상남도 진주시 하대동(대성맨션 부근),3
2,경상남도 통영시 광도면(죽림주공아파트정문 부근),3
3,대구광역시 수성구 지산동(에덴테마어린이공원 부근),3
4,대구광역시 수성구 파동(서민할매식당 부근),3


In [15]:
t3_1['서울'] = t3_1['다발지명'].str.split(' ').str[0]
t3_1 = t3_1[t3_1['서울'] == '서울특별시']
t3_1
# t3_1.info()

C:\Users\LG\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,다발지명,발생건수,서울
33,서울특별시 강북구 미아동(삼양시장오거리 부근),4,서울특별시
34,서울특별시 노원구 상계동(홍파복지원앞사거리 부근),3,서울특별시
35,서울특별시 노원구 중계동(중계그린아파트107동 부근),3,서울특별시
42,서울특별시 서대문구 북가좌동(북가좌초교사거리 부근),3,서울특별시
43,서울특별시 양천구 신정동(수정아파트102동 부근),3,서울특별시
44,서울특별시 서초구 우면동(서초네이처힐 3단지 314동 부근),3,서울특별시
49,서울특별시 서초구 방배동(방배초교입구사거리 부근),3,서울특별시
50,서울특별시 강남구 도곡동(언주초교 부근),3,서울특별시
51,서울특별시 강남구 역삼동(상지리츠빌1차 부근),3,서울특별시
55,서울특별시 송파구 방이동(방이초교 부근),4,서울특별시


In [16]:
t3_1['서울'] = t3_1['다발지명'].str.split(' ').str[0]
t3_1 = t3_1[t3_1['서울'] == '서울특별시'] 
t3_1['구'] = t3_1['다발지명'].str.split(' ').str[1]
t3_1['동-'] = t3_1['다발지명'].str.split(' ').str[2]
t3_1['동'] = t3_1['동-'].str.split('(').str[0]
t3_1 = t3_1[["구", "동", "발생건수"]] 
t3_1

,구,동,발생건수
33,강북구,미아동,4
34,노원구,상계동,3
35,노원구,중계동,3
42,서대문구,북가좌동,3
43,양천구,신정동,3
44,서초구,우면동,3
49,서초구,방배동,3
50,강남구,도곡동,3
51,강남구,역삼동,3
55,송파구,방이동,4


In [17]:
t3_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 33 to 56
Data columns (total 3 columns):
구       11 non-null object
동       11 non-null object
발생건수    11 non-null int64
dtypes: int64(1), object(2)
memory usage: 352.0+ bytes


In [21]:
# 컬럼이름 변경
t3_1.rename(columns={
    t3_1.columns[2]:"어린이사고발생건수"
}, inplace=True)

t3_1.head()

,구,동,어린이사고발생건수
33,강북구,미아동,4
34,노원구,상계동,3
35,노원구,중계동,3
42,서대문구,북가좌동,3
43,양천구,신정동,3


## merge

In [27]:
mt = pd.merge(t2_2,t3_1, on=["구","동"], how='outer') #full outer: 양쪽에 있는것을 다가져옴
mt.head()

,구,동,노인사고발생건수,어린이사고발생건수
0,강남구,도곡동,3.00,3.00
1,강동구,고덕동,3.00,nan
2,강동구,길동,4.00,nan
3,강동구,명일동,10.00,nan
4,강동구,성내동,3.00,nan
